In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parâmetros do problema
nx, ny = 100, 100
dx, dy = 1, 1
alpha = 0.02
omega = 1 / (3 * alpha + 0.5)
rho0 = 1
u_max = 0.1

# Componentes de velocidade e pesos
cx = np.array([0, 1, 0, -1, 0, 1, -1, -1, 1])
cy = np.array([0, 0, 1, 0, -1, 1, 1, -1, -1])
w = np.array([0.4444444444, 0.1111111111, 0.1111111111, 0.1111111111, 
              0.1111111111, 0.0277777777, 0.0277777777, 0.0277777777, 0.0277777777])

# Definição de malha
X = np.linspace(0.5, 0.5 + (nx - 1) * dx, nx)
Y = np.linspace(0.5, 0.5 + (ny - 1) * dy, ny)
kx, ky = 2 * np.pi, 2 * np.pi    ## estamos considerando l=1
ryx = np.sqrt(ky / kx)
rxy = np.sqrt(kx / ky)

# Inicialização das variáveis
f = np.zeros((9, nx, ny))
fp = np.zeros((9, nx, ny))
u = np.zeros((nx, ny))
v = np.zeros((nx, ny))
rho = np.zeros((nx, ny))
p = np.zeros((nx, ny))

# Inicializando rho
for x in range(nx):
    for y in range(ny):
        rho[x, y] = rho0 + 3 * (-0.25 * rho0 * u_max**2 * ((ky / kx) * np.cos(2 * kx * X[x]) + (kx / ky) * np.cos(2 * ky * Y[y])))

# Inicializando u e v
for x in range(nx):
    for y in range(ny):
        u[x, y] = u_max * (-ryx * np.cos(kx * X[x]) * np.sin(ky * Y[y]))
        v[x, y] = u_max * rxy * np.sin(kx * X[x]) * np.cos(ky * Y[y])

# Inicializando f
for x in range(nx):
    for y in range(ny):
        t1 = u[x, y]**2 + v[x, y]**2
        for i in range(9):
            t2 = u[x, y] * cx[i] + v[x, y] * cy[i]
            f[i, x, y] = w[i] * rho[x, y] * (1 + 3 * t2 + 4.5 * t2**2 - 1.5 * t1)

# Funções de colisão e transmissão
def fcolisao(u, v, f, fp, rho):
    for x in range(nx):
        for y in range(ny):
            t1 = u[x, y]**2 + v[x, y]**2
            for i in range(9):
                t2 = u[x, y] * cx[i] + v[x, y] * cy[i]
                feq = w[i] * rho[x, y] * (1 + 3 * t2 + 4.5 * t2**2 - 1.5 * t1)
                fp[i, x, y] = f[i, x, y] * (1 - omega) + omega * feq

def ftransmissao(f, fp):
    for i in range(9):
        for x in range(nx):
            for y in range(ny):
                xx = (x + int(cx[i])) % nx
                yy = (y + int(cy[i])) % ny
                f[i, xx, yy] = fp[i, x, y]

# Loop temporal
for count in range(3000):
    # Colisão
    fcolisao(u, v, f, fp, rho)
    
    # Transmissão
    ftransmissao(f, fp)
    
    # Atualização de rho e p
    rho = f.sum(axis=0)
    p = rho / 3
    
    # Atualização de u e v
    u = (f[1] + f[5] + f[8] - (f[3] + f[6] + f[7])) / rho
    v = (f[2] + f[5] + f[6] - (f[4] + f[7] + f[8])) / rho

# Índice y fixado na metade do range
y_fixed = ny // 2

# Valores de u ao longo de x para o y fixo
u_slice = u[:, y_fixed]

# Plotando o gráfico
plt.figure(figsize=(10, 6))
plt.plot(range(nx), u_slice, label=f'u em y={y_fixed}', color='b', marker='o')
plt.xlabel('Posição x')
plt.ylabel('Velocidade u')
plt.title(f'Perfil de u ao longo de x para y = {y_fixed}')
plt.legend()
plt.grid(True)
plt.show()
